In [2]:
import pandas as pd
import requests
from dotenv import load_dotenv
import os
import io
import duckdb

In [3]:
## Load dotenv variables
load_dotenv()

#Initialize duckdb with postgres connector    
cursor = duckdb.connect()
cursor.sql("INSTALL postgres;")
cursor.sql("LOAD postgres;")
cursor.sql(f"""
           ATTACH 'dbname=youthbase user={os.getenv("PG_SPORTANALYTICS_USER")} password={os.getenv("PG_SPORTANALYTICS_PASSWORD")} 
           host={os.getenv("PG_SPORTANALYTICS_HOST")}' AS postgres_db (TYPE POSTGRES);
           """)

In [66]:
## Get all file_ids from bucket

# Define directory URL and create empty file id list
dir_uri = f'{os.getenv("kd_dns")}3/drive/{os.getenv("kd_id")}/files/4777/files'
headers = {'Authorization': 'Bearer ' + os.getenv("kd_token")}
file_ids = []

# Return results from paginated API request
response = requests.get(dir_uri, headers=headers)
content = response.json()

# Append the multiple returned file ids
for i in content['data']:
    file_ids.append(i['id'])

# Loop through the pagination token
# Same cursor token is applied as long as has_more variable is True
cursor = content['cursor']
has_more = content['has_more']
while has_more == True:
    loop_resp = requests.get(dir_uri + '?cursor=' + cursor, headers=headers)
    loop_cont = loop_resp.json()
    # print(loop_cont['cursor'])

    for i in loop_cont['data']:
        file_ids.append(i['id'])

    has_more = loop_cont['has_more']
    cursor = loop_cont['cursor']

KeyError: 'data'

In [ ]:
df = pd.DataFrame()
for id in file_ids:
    file_url = f'{os.getenv("kd_dns")}/2/drive/{os.getenv("kd_id")}/files/{id}/download'
    # Load table from kDrive directory into a raw DataFrame
    response = requests.get(file_url, headers=headers)
    raw_df = pd.read_excel(io.BytesIO(response.content), engine='openpyxl')

    # Remove first frow which subtitle from dataframe
    raw_df = raw_df.loc[1:]

    # Unpivot Table to set set a row for a measure
    raw_df = pd.melt(raw_df, id_vars=raw_df.columns[:6], value_vars=raw_df.columns[6:])
    raw_df.dropna(subset=['value'], inplace=True)
    raw_df = raw_df[raw_df['value'] != 0]
    raw_df = raw_df[raw_df['variable'] != 'Comment']
    raw_df['value'] = raw_df['value'].astype(float)
    raw_df.columns = map(str.lower, raw_df.columns)

    # Append raw DataFrame into final DataFrame
    df = pd.concat([df, raw_df], ignore_index=True)
    # convert birthday column to just date
    df['birthday'] = pd.to_datetime(df['birthday']).dt.date

In [67]:
df.head()

Unnamed: 0_level_0   Unnamed: 1_level_0     Unnamed: 2_level_0  \
  Unnamed: 0_level_1   Unnamed: 1_level_1     Unnamed: 2_level_1   
0          Luan Jaun  BSC Young Boys, U15  Jul 22, 2024 17:00:00   
1          Luan Jaun  BSC Young Boys, U15  Jul 22, 2024 17:00:00   
2          Luan Jaun  BSC Young Boys, U15  Jul 22, 2024 17:00:00   
3          Luan Jaun  BSC Young Boys, U15  Jul 23, 2024 10:00:00   
4          Luan Jaun  BSC Young Boys, U15  Jul 24, 2024 17:00:00   

  Unnamed: 3_level_0     Unnamed: 4_level_0 Unnamed: 5_level_0  \
  Unnamed: 3_level_1     Unnamed: 4_level_1 Unnamed: 5_level_1   
0                NaN  TH Training U15 / U16    THT DZ auf BACK   
1                NaN  TH Training U15 / U16   THT Einstieg SAB   
2                NaN  TH Training U15 / U16   THT RZ im Winkel   
3     BSC Young Boys          Training U-15             THT RZ   
4     BSC Young Boys          Training U-15            THT BiT   

  Unnamed: 6_level_0   Parent Category Id Verhalten        ...        \
  Unnamed: 6_level_1        Category Name      BACK CATCH  ...  STAY   
0                NaN  4 Etwas anstrengend      15.0   NaN  ...   NaN   
1                NaN             3 Mässig       NaN   NaN  ...   NaN   
2                NaN  4 Etwas anstrengend       NaN   NaN  ...  15.0   
3                NaN               0 Ruhe      45.0   NaN  ...  45.0   
4                NaN                    6      45.0  45.0  ...   NaN   

                DEF                                                \
  Ball in die Tiefe Distanzschuss Hohe Bälle Nahdistanz Rote Zone   
0               NaN          15.0        NaN        NaN       NaN   
1               NaN           NaN        NaN        NaN       NaN   
2               NaN           NaN        NaN        NaN      15.0   
3               NaN           NaN        NaN        NaN      45.0   
4              45.0           NaN        NaN        NaN       NaN   

                                            OFF                       \
  Rück-/Querpassverteidigung Spielaufbau Zone 1 Spielaufbau Zone 2/3   
0                        NaN                NaN                  NaN   
1                        NaN               15.0                  NaN   
2                        NaN                NaN                  NaN   
3                        NaN                NaN                  NaN   
4                        NaN                NaN                  NaN   

                      
  Umschalten DEF-OFF  
0                NaN  
1                NaN  
2                NaN  
3                NaN  
4                NaN  

[5 rows x 21 columns]

In [5]:
table_name = 'anthrophometrie'
cursor.sql(f"DROP TABLE IF EXISTS postgres_db.{table_name};")
# cursor.sql(f"CREATE TABLE IF NOT EXISTS postgres_db.{table_name} AS SELECT * FROM df;")

# Create a goalkeeper Dimension with current Team

In [19]:
goalies = pd.read_csv('/Users/matthiashugli/Downloads/keeper-export.csv', sep=',')
goalies['Player Name'] = goalies['Vorname'] + ' ' + goalies['Name']
goalies['Initialen'] = goalies['Vorname'].astype(str).str[0] + goalies['Name'].astype(str).str[0]
goalies['Geburtsdatum'] = pd.to_datetime(goalies['Geburtsdatum'], dayfirst=True)

table_name = 'goalies'
cursor.sql(f"DROP TABLE IF EXISTS postgres_db.{table_name};")
cursor.sql(f"CREATE TABLE IF NOT EXISTS postgres_db.{table_name} AS SELECT * FROM goalies")

# YB Data

In [6]:
df = pd.read_csv('/Users/matthiashugli/Downloads/s1_core_trm_player_unit_skills.csv', sep=';', header=[1,2], skiprows=[3])
df.head()

Unnamed: 0_level_0   Unnamed: 1_level_0     Unnamed: 2_level_0  \
  Unnamed: 0_level_1   Unnamed: 1_level_1     Unnamed: 2_level_1   
0          Luan Jaun  BSC Young Boys, U15  Jul 22, 2024 17:00:00   
1          Luan Jaun  BSC Young Boys, U15  Jul 22, 2024 17:00:00   
2          Luan Jaun  BSC Young Boys, U15  Jul 22, 2024 17:00:00   
3          Luan Jaun  BSC Young Boys, U15  Jul 23, 2024 10:00:00   
4          Luan Jaun  BSC Young Boys, U15  Jul 24, 2024 17:00:00   

  Unnamed: 3_level_0     Unnamed: 4_level_0 Unnamed: 5_level_0  \
  Unnamed: 3_level_1     Unnamed: 4_level_1 Unnamed: 5_level_1   
0                NaN  TH Training U15 / U16    THT DZ auf BACK   
1                NaN  TH Training U15 / U16   THT Einstieg SAB   
2                NaN  TH Training U15 / U16   THT RZ im Winkel   
3     BSC Young Boys          Training U-15             THT RZ   
4     BSC Young Boys          Training U-15            THT BiT   

  Unnamed: 6_level_0   Parent Category Id Verhalten        ...        \
  Unnamed: 6_level_1        Category Name      BACK CATCH  ...  STAY   
0                NaN  4 Etwas anstrengend      15.0   NaN  ...   NaN   
1                NaN             3 Mässig       NaN   NaN  ...   NaN   
2                NaN  4 Etwas anstrengend       NaN   NaN  ...  15.0   
3                NaN               0 Ruhe      45.0   NaN  ...  45.0   
4                NaN                    6      45.0  45.0  ...   NaN   

                DEF                                                \
  Ball in die Tiefe Distanzschuss Hohe Bälle Nahdistanz Rote Zone   
0               NaN          15.0        NaN        NaN       NaN   
1               NaN           NaN        NaN        NaN       NaN   
2               NaN           NaN        NaN        NaN      15.0   
3               NaN           NaN        NaN        NaN      45.0   
4              45.0           NaN        NaN        NaN       NaN   

                                            OFF                       \
  Rück-/Querpassverteidigung Spielaufbau Zone 1 Spielaufbau Zone 2/3   
0                        NaN                NaN                  NaN   
1                        NaN               15.0                  NaN   
2                        NaN                NaN                  NaN   
3                        NaN                NaN                  NaN   
4                        NaN                NaN                  NaN   

                      
  Umschalten DEF-OFF  
0                NaN  
1                NaN  
2                NaN  
3                NaN  
4                NaN  

[5 rows x 21 columns]

In [401]:
df = pd.read_csv('/Users/matthiashugli/Downloads/s1_core_trm_player_unit_skills.csv', sep=';')
df.columns = df.iloc[2, :8].tolist() + (df.iloc[0, 8:] + ',' + df.iloc[1, 8:]).tolist()
df = df.iloc[3:]
melt_df = df.melt(id_vars=df.columns[:8], value_vars=df.iloc[8:]) \
            .dropna(subset=['value']) \
            .reset_index()

melt_df.columns = melt_df.columns.str.replace(" ", "_")
melt_df[['verhalten', 'entscheid']] = melt_df['variable'].str.split(',', expand=True)
cursor.sql("DROP TABLE IF EXISTS skills_def; CREATE TEMPORARY TABLE skills_def AS SELECT * FROM melt_df;")

In [402]:
melt_df.head()

,index,Player,Player's_current_team,Training_Start_Datetime,Training_Team_Name,Training_Text,Training_Unit_Short_Text,Training_Unit_Description,Training_Unit_Intensity_Text,variable,value,verhalten,entscheid
0,0,Luan Jaun,"BSC Young Boys, U15","Jul 22, 2024 17:00:00",NaN,TH Training U15 / U16,THT DZ auf BACK,NaN,4 Etwas anstrengend,"Verhalten,BACK",15,Verhalten,BACK
1,3,Luan Jaun,"BSC Young Boys, U15","Jul 23, 2024 10:00:00",BSC Young Boys,Training U-15,THT RZ,NaN,0 Ruhe,"Verhalten,BACK",45,Verhalten,BACK
2,4,Luan Jaun,"BSC Young Boys, U15","Jul 24, 2024 17:00:00",BSC Young Boys,Training U-15,THT BiT,NaN,6,"Verhalten,BACK",45,Verhalten,BACK
3,5,Luan Jaun,"BSC Young Boys, U15","Jul 30, 2024 17:00:00",BSC Young Boys,Training U-15,THT Rote Zone,- Orientierung im Winkel,0 Ruhe,"Verhalten,BACK",15,Verhalten,BACK
4,7,Luan Jaun,"BSC Young Boys, U15","Jul 31, 2024 10:00:00",BSC Young Boys,Training U-15,THT RQPV,NaN,0 Ruhe,"Verhalten,BACK",40,Verhalten,BACK


In [403]:
cursor.sql("""SELECT sps.Index
           ,sps.Player AS Keeper
           ,CAST(strptime(sps.Training_Start_Datetime, '%b %-j, %Y %H:%M:%S') AS DATE) AS Datum
           ,CAST(strptime(sps.Training_Start_Datetime, '%b %-j, %Y %H:%M:%S') AS TIMESTAMP) AS Zeit
           ,sps.Training_Team_Name AS Trainingsteam
           ,sps.Training_Text AS Titel
           ,sps.Training_Unit_Short_Text AS Übung
           ,sps.Training_Unit_Description AS Notizen
           ,sps.Training_Unit_Intensity_Text AS Intensität
           ,sps.verhalten AS Kategorie
           ,sps.entscheid AS Skill
           ,'Spielaufbau' AS Entscheid
           ,CAST(sps.value AS INT) AS Dauer
           FROM skills_def sps
           WHERE sps.verhalten = 'OFF';           
           """)

┌───────┬──────────────────┬────────────┬───┬───────────┬────────────────────┬─────────────┬───────┐
│ index │      Keeper      │   Datum    │ … │ Kategorie │       Skill        │  Entscheid  │ Dauer │
│ int64 │     varchar      │    date    │   │  varchar  │      varchar       │   varchar   │ int32 │
├───────┼──────────────────┼────────────┼───┼───────────┼────────────────────┼─────────────┼───────┤
│  4371 │ Luan Jaun        │ 2024-01-22 │ … │ OFF       │ Spielaufbau Zone 1 │ Spielaufbau │    15 │
│  4376 │ Luan Jaun        │ 2024-01-30 │ … │ OFF       │ Spielaufbau Zone 1 │ Spielaufbau │    35 │
│  4388 │ Luan Jaun        │ 2024-01-16 │ … │ OFF       │ Spielaufbau Zone 1 │ Spielaufbau │    15 │
│  4391 │ Luan Jaun        │ 2024-01-25 │ … │ OFF       │ Spielaufbau Zone 1 │ Spielaufbau │    25 │
│  4396 │ Denis Martinovic │ 2024-01-17 │ … │ OFF       │ Spielaufbau Zone 1 │ Spielaufbau │    30 │
│  4400 │ Denis Martinovic │ 2024-01-22 │ … │ OFF       │ Spielaufbau Zone 1 │ Spielaufbau 

In [404]:
cursor.sql("""SELECT sps.Index
           ,sps.Player AS Keeper
           ,CAST(strptime(sps.Training_Start_Datetime, '%b %-j, %Y %H:%M:%S') AS DATE) AS Datum
           ,CAST(strptime(sps.Training_Start_Datetime, '%b %-j, %Y %H:%M:%S') AS TIMESTAMP) AS Zeit
           ,sps.Training_Team_Name AS Trainingsteam
           ,sps.Training_Text AS Titel
           ,sps.Training_Unit_Short_Text AS Übung
           ,sps.Training_Unit_Description AS Notizen
           ,sps.Training_Unit_Intensity_Text AS Intensität
           ,sps.verhalten AS Kategorie
           ,sps.entscheid AS Skill
           ,ent.entscheid AS Entscheid
           ,CAST(sps.value AS INT) AS Dauer
           FROM skills_def sps
           LEFT JOIN (SELECT * FROM skills_def WHERE verhalten = 'Verhalten') ent 
           ON sps.Player = ent.Player
           AND sps.Training_Start_Datetime = ent.Training_Start_Datetime
           AND sps.Training_Unit_Short_Text = ent.Training_Unit_Short_Text
           WHERE sps.verhalten = 'DEF'
           
           UNION ALL
           
           SELECT sps.Index
           ,sps.Player AS Keeper
           ,CAST(strptime(sps.Training_Start_Datetime, '%b %-j, %Y %H:%M:%S') AS DATE) AS Datum
           ,CAST(strptime(sps.Training_Start_Datetime, '%b %-j, %Y %H:%M:%S') AS TIMESTAMP) AS Zeit
           ,sps.Training_Team_Name AS Trainingsteam
           ,sps.Training_Text AS Titel
           ,sps.Training_Unit_Short_Text AS Übung
           ,sps.Training_Unit_Description AS Notizen
           ,sps.Training_Unit_Intensity_Text AS Intensität
           ,sps.verhalten AS Kategorie
           ,sps.entscheid AS Skill
           ,'Spielaufbau' AS Entscheid
           ,CAST(sps.value AS INT) AS Dauer
           FROM skills_def sps
           WHERE sps.verhalten = 'OFF';           
           """)


# ['Keeper', 'Team', 'DatumZeit', 'Trainingsteam', 'Titel', 'Übung', 'Notizen', 'Intensität', 'Kategorie', 'Skill', 'Dauer']

┌───────┬──────────────────┬────────────┬───┬───────────┬────────────────────┬─────────────┬───────┐
│ index │      Keeper      │   Datum    │ … │ Kategorie │       Skill        │  Entscheid  │ Dauer │
│ int64 │     varchar      │    date    │   │  varchar  │      varchar       │   varchar   │ int32 │
├───────┼──────────────────┼────────────┼───┼───────────┼────────────────────┼─────────────┼───────┤
│  1752 │ Luan Jaun        │ 2024-01-24 │ … │ DEF       │ Ball in die Tiefe  │ GO          │    45 │
│  1776 │ Denis Martinovic │ 2024-01-21 │ … │ DEF       │ Ball in die Tiefe  │ GO          │    30 │
│  1777 │ Denis Martinovic │ 2024-01-22 │ … │ DEF       │ Ball in die Tiefe  │ STAY        │    30 │
│  1783 │ Denis Martinovic │ 2024-01-26 │ … │ DEF       │ Ball in die Tiefe  │ STAY        │    25 │
│  1800 │ Denis Martinovic │ 2024-01-02 │ … │ DEF       │ Ball in die Tiefe  │ CATCH       │    65 │
│  1801 │ Denis Martinovic │ 2024-01-03 │ … │ DEF       │ Ball in die Tiefe  │ GO          

In [311]:
df = pd.read_csv('/Users/matthiashugli/Downloads/s1_core_trm_player_unit_skills.csv', sep=';', header=[1,2], skiprows=[3])
df \
            .set_index([('Unnamed: 0_level_0', 'Unnamed: 0_level_1'), ('Unnamed: 1_level_0', 'Unnamed: 1_level_1'), 
                        ('Unnamed: 2_level_0', 'Unnamed: 2_level_1'), ('Unnamed: 3_level_0', 'Unnamed: 3_level_1'), 
                        ('Unnamed: 4_level_0', 'Unnamed: 4_level_1'), ('Unnamed: 5_level_0', 'Unnamed: 5_level_1'),
                         ('Unnamed: 6_level_0', 'Unnamed: 6_level_1'), ('Parent Category Id', 'Category Name')])

Verhalten  \
                                                                                                                                                                                                                                                                                                                                        BACK   
(Unnamed: 0_level_0, Unnamed: 0_level_1) (Unnamed: 1_level_0, Unnamed: 1_level_1) (Unnamed: 2_level_0, Unnamed: 2_level_1) (Unnamed: 3_level_0, Unnamed: 3_level_1) (Unnamed: 4_level_0, Unnamed: 4_level_1) (Unnamed: 5_level_0, Unnamed: 5_level_1) (Unnamed: 6_level_0, Unnamed: 6_level_1) (Parent Category Id, Category Name)             
Jara Ackermann                           ♀ BSC YB-Frauen                          Sep 4, 2024 10:30:00                     BSC Young Boys                           Individual Training                      TH BiT                                   NaN                                      4 Etwas anstrengend                       NaN   
                                                                                                                                                                                                             TH DS                                    NaN                                      4 Etwas anstrengend                      15.0   
                                                                                  Sep 11, 2024 10:30:00                    BSC Young Boys                           Individual Training                      TH BiT                                   NaN                                      4 Etwas anstrengend                       NaN   
                                                                                                                                                                                                             TH DS                                    NaN                                      4 Etwas anstrengend                      15.0   
                                                                                  Sep 18, 2024 10:30:00                    BSC Young Boys                           Individual Training                      TH BiT                                   NaN                                      4 Etwas anstrengend                       NaN   
...                                                                                                                                                                                                                                                                                                                                      ...   
Felix Stuber                             BSC Young Boys, U16                      Sep 16, 2024 17:30:00                    NaN                                      THT OFF                                  THT OFF Z1                               NaN                                      3 Mässig                                  NaN   
                                                                                  Sep 18, 2024 17:30:00                    BSC Young Boys                           Training U-16                            TH HB                                    NaN                                      4 Etwas anstrengend                      30.0   
                                                                                  Sep 19, 2024 17:00:00                    NaN                                      THT OFF                                  TH OFF Z1                                NaN                                      3 Mässig                                  NaN   
                                                                                  Sep 23, 2024 16:15:00                    NaN                                      TH Training U15/16                       TH RQPV                                  NaN                                      4 Et

In [7]:
df = pd.read_csv('/Users/matthiashugli/Downloads/s1_core_trm_player_unit_skills.csv', sep=';', header=[1,2], skiprows=[3])
melt_df = df \
            .set_index([('Unnamed: 0_level_0', 'Unnamed: 0_level_1'), ('Unnamed: 1_level_0', 'Unnamed: 1_level_1'), 
                        ('Unnamed: 2_level_0', 'Unnamed: 2_level_1'), ('Unnamed: 3_level_0', 'Unnamed: 3_level_1'), 
                        ('Unnamed: 4_level_0', 'Unnamed: 4_level_1'), ('Unnamed: 5_level_0', 'Unnamed: 5_level_1'),
                         ('Unnamed: 6_level_0', 'Unnamed: 6_level_1'), ('Parent Category Id', 'Category Name')]) \
            .melt(ignore_index=False) \
            .dropna(subset=['value']) \
            .reset_index()

melt_df.columns = ['Keeper', 'Team', 'DatumZeit', 'Trainingsteam', 'Titel', 'Übung', 'Notizen', 'Intensität', 'Kategorie', 'Skill', 'Dauer']
# Split skills and decisions into own DataFrames and concat on the column axis
def_df = melt_df[melt_df['Kategorie'] == 'DEF']
verhalten_df = melt_df[melt_df['Kategorie'] == 'Verhalten']
defensive_skills = verhalten_df.merge(def_df, left_on=['Keeper', 'DatumZeit', 'Übung'], right_on=['Keeper', 'DatumZeit', 'Übung'], suffixes=[None, 'DEF']).reset_index()

defensive_skills = defensive_skills[['Keeper', 'Team', 'DatumZeit', 'Trainingsteam', 'Titel', 'Übung',
                        'Notizen', 'Intensität', 'Skill', 'KategorieDEF', 'SkillDEF', 'Dauer']].copy()
defensive_skills = defensive_skills.rename(columns={'Skill': 'Verhalten', 'KategorieDEF': 'DEF/OFF', 'SkillDEF': 'Spielsituation'})

# Filter the offensive content
off_df = melt_df[melt_df['Kategorie'] == 'OFF']
off_df = off_df.rename(columns={'Kategorie': 'DEF/OFF', 'Skill': 'Spielsituation'})

# Concat to the defensive DataFrame on the row axis
th_skills = pd.concat([defensive_skills, off_df], sort=False)

# Convert columns to their data type
th_skills['Datum'] = pd.to_datetime(th_skills.DatumZeit).dt.date
th_skills['Zeit'] = pd.to_datetime(th_skills.DatumZeit).dt.time
th_skills.drop(columns=['DatumZeit'], inplace=True)

# Export DataFrame to CSV
th_skills.to_csv('/Users/matthiashugli/kDrive/goalkeeping/💛🖤 YB Nachwuchs/csv_th_skill.csv', index=False, sep=";")
#df[df.iloc[:, 0] == 'Marco Iseli']
th_skills[th_skills['Keeper'] == 'Nik Lyoth']

,Keeper,Team,Trainingsteam,Titel,Übung,Notizen,Intensität,Verhalten,DEF/OFF,Spielsituation,Dauer,Datum,Zeit
55,Nik Lyoth,"FE-14 PSA Bern, U14",NaN,TH Training FE14,THT RZ (Winkel / Zentral),NaN,6,BACK,DEF,Rote Zone,90.0,2024-07-29,09:00:00
56,Nik Lyoth,"FE-14 PSA Bern, U14",BSC Young Boys,Training U-15,THT RQPV,NaN,0 Ruhe,BACK,DEF,Rück-/Querpassverteidigung,40.0,2024-07-31,10:00:00
57,Nik Lyoth,"FE-14 PSA Bern, U14",NaN,TH Training FE14 - TL,THT RQPV,NaN,6,BACK,DEF,Rück-/Querpassverteidigung,75.0,2024-08-06,15:00:00
58,Nik Lyoth,"FE-14 PSA Bern, U14",NaN,TH Training FE14 / U15 - TL,THT FE14 RZ,NaN,4 Etwas anstrengend,BACK,DEF,Rote Zone,35.0,2024-08-07,15:00:00
59,Nik Lyoth,"FE-14 PSA Bern, U14",NaN,TH Training FE14,THT RQPV,NaN,5 Anstrengend,BACK,DEF,Rück-/Querpassverteidigung,75.0,2024-08-12,17:00:00
60,Nik Lyoth,"FE-14 PSA Bern, U14",BSC Young Boys,Training U-15,THT RZ / RQPV,NaN,4 Etwas anstrengend,BACK,DEF,Rote Zone,60.0,2024-08-13,17:00:00
61,Nik Lyoth,"FE-14 PSA Bern, U14",BSC Young Boys,Training U-15,THT Hohe Bälle (Spielform Flankenspiel),NaN,5 Anstrengend,BACK,DEF,Hohe Bälle,45.0,2024-08-14,17:00:00
62,Nik Lyoth,"FE-14 PSA Bern, U14",NaN,TH Training FE14,THT Distanzschuss,Hechten one-step,5 Anstrengend,BACK,DEF,Distanzschuss,30.0,2024-08-26,17:00:00
63,Nik Lyoth,"FE-14 PSA Bern, U14",NaN,THT + Integriert FE14 / U15,THT RQPV mit organisiertem Block,NaN,4 Etwas anstrengend,BACK,DEF,Rück-/Querpassverteidigung,60.0,2024-08-27,17:00:00
64,Nik Lyoth,"FE-14 PSA Bern, U14",NaN,TH Training FE14,THT Hohe Bälle,NaN,0 Ruhe,BACK,DEF,Hohe Bälle,90.0,2024-09-09,17:00:00


In [8]:
table_name = 'trainings'
cursor.sql(f"DROP TABLE IF EXISTS postgres_db.{table_name};")
cursor.sql(f"CREATE TABLE IF NOT EXISTS postgres_db.{table_name} AS SELECT * FROM th_skills")

## Anwesenheiten

In [456]:
df = pd.read_csv('/Users/matthiashugli/Downloads/s1_CORE_TRAINING_PARTICIPANTS_V2_CV.csv', sep=';')
df.columns = df.columns.str.replace(" ", "_")
df.columns = df.columns.str.replace(":", "")
df['Stufe'] = [x[-1] for x in df['Team'].str.split(', ')]
df['Datum'] = pd.to_datetime(df.Training_Start_Datetime).dt.date
df['Zeit'] = pd.to_datetime(df.Training_Start_Datetime).dt.time
#df.drop(columns=['Training_Start_Datetime'], inplace=True)
df.head()

,Person,Position,Team,Training_Text,Training_Team,Training_Start_Datetime,Abscence_Reason,Unnamed_7,Stufe,Datum,Zeit
0,Jan Schmid,Tor,"FE-13 BSC YOUNG BOYS black, U13",TH Training FE13,NaN,"Jul 31, 2024 15:15:00",Privat,0,U13,2024-07-31,15:15:00
1,Jan Schmid,Tor,"FE-13 BSC YOUNG BOYS black, U13",TH Training FE13,NaN,"Aug 28, 2024 3:00:00",Privat,0,U13,2024-08-28,03:00:00
2,Jan Schmid,Tor,"FE-13 BSC YOUNG BOYS black, U13",TH Training FE13,NaN,"Sep 4, 2024 15:00:00",NaN,1,U13,2024-09-04,15:00:00
3,Jan Schmid,Tor,"FE-13 BSC YOUNG BOYS black, U13",TH Training FE13,NaN,"Sep 11, 2024 9:00:00",NaN,1,U13,2024-09-11,09:00:00
4,Jan Schmid,Tor,"FE-13 BSC YOUNG BOYS black, U13",TH Training FE13,NaN,"Sep 18, 2024 9:00:00",NaN,1,U13,2024-09-18,09:00:00


In [464]:
# cursor.sql("SELECT Training_Start_Datetime FROM df (TIMESTAMPFORMAT '%b %-j, %Y %h:%M:%S')")
# cursor.sql("SELECT strptime(Training_Start_Datetime, '%b %-j, %Y %h:%M:%S') from df")
cursor.sql("SELECT try_strptime(Training_Start_Datetime, '%b %-j, %Y %h:%M:%S') from df")

┌──────────────────────────────────────────────────────────────┐
│ try_strptime(Training_Start_Datetime, '%B %-j, %Y %h:%M:%S') │
│                          timestamp                           │
├──────────────────────────────────────────────────────────────┤
│ NULL                                                         │
│ NULL                                                         │
│ NULL                                                         │
│ NULL                                                         │
│ NULL                                                         │
│ NULL                                                         │
│ NULL                                                         │
│ NULL                                                         │
│ NULL                                                         │
│ NULL                                                         │
│  ·                                                           │
│  ·                     

In [443]:
cursor.sql("""
            SELECT Person AS Keeper
             ,CAST(Datum AS DATE) AS Datum
            ,CAST(Zeit AS TIME) AS Zeit
            ,Position AS Position
            ,Team AS Team
            ,Stufe AS Stufe
            ,Training_Text AS Titel
            ,Training_Team AS Trainingsteam
            ,Abscence_Reason AS Abwesenheitsgrund
            ,CAST(Unnamed_7 AS INT) AS Anwesend
            FROM df limit 10;
           """)

# df[['Keeper', 'Position', 'Team', 'Stufe', 'Titel', 'Trainingsteam', 'Datum', 'Abwesenheitsgrund', 'Anwesend']]

┌────────────┬────────────┬──────────┬──────────┬───┬──────────────────┬───────────────┬───────────────────┬──────────┐
│   Keeper   │   Datum    │   Zeit   │ Position │ … │      Titel       │ Trainingsteam │ Abwesenheitsgrund │ Anwesend │
│  varchar   │    date    │   time   │ varchar  │   │     varchar      │    varchar    │      varchar      │  int32   │
├────────────┼────────────┼──────────┼──────────┼───┼──────────────────┼───────────────┼───────────────────┼──────────┤
│ Jan Schmid │ 2024-07-31 │ 15:15:00 │ Tor      │ … │ TH Training FE13 │ NULL          │ Privat            │        0 │
│ Jan Schmid │ 2024-08-28 │ 03:00:00 │ Tor      │ … │ TH Training FE13 │ NULL          │ Privat            │        0 │
│ Jan Schmid │ 2024-09-04 │ 15:00:00 │ Tor      │ … │ TH Training FE13 │ NULL          │ NULL              │        1 │
│ Jan Schmid │ 2024-09-11 │ 09:00:00 │ Tor      │ … │ TH Training FE13 │ NULL          │ NULL              │        1 │
│ Jan Schmid │ 2024-09-18 │ 09:00:00 │ T

In [25]:
df = pd.read_csv('/Users/matthiashugli/Downloads/s1_CORE_TRAINING_PARTICIPANTS_V2_CV.csv', sep=';')
df['Stufe'] = [x[-1] for x in df['Team'].str.split(', ')]

# Convert columns to their data type
df['Datum'] = pd.to_datetime(df.Datum).dt.date
participation = df[['Keeper', 'Position', 'Team', 'Stufe', 'Titel', 'Trainingsteam', 'Datum', 'Abwesenheitsgrund', 'Anwesend']]
participation.head()

table_name = 'anwesenheiten'
cursor.sql(f"DROP TABLE IF EXISTS postgres_db.{table_name};")
cursor.sql(f"CREATE TABLE IF NOT EXISTS postgres_db.{table_name} AS SELECT * FROM participation")

# Date Table

In [444]:
cursor.sql("""
            WITH date_range AS (
                SELECT
                    MIN(Datum) AS start_date,
                    MAX(Datum) AS end_date
                FROM th_skills
            ),
            dates AS (
                SELECT
                    start_date + INTERVAL '1 day' * (ROW_NUMBER() OVER (ORDER BY seq) - 1) AS date
                FROM
                    (SELECT start_date, end_date FROM date_range) AS dr
                JOIN
                    (SELECT seq FROM range(0, (SELECT EXTRACT(day FROM end_date - start_date) FROM date_range))) AS r ON true
            )
            SELECT
                date,
                EXTRACT(year FROM date) AS year,
                EXTRACT(month FROM date) AS month,
                TO_CHAR(date, 'Month') AS month_name,
                EXTRACT(day FROM date) AS day,
                TO_CHAR(date, 'Day') AS day_name,
                EXTRACT(doy FROM date) AS day_number,
                TO_CHAR(date, 'YYYY-MM') AS year_month
            FROM dates
            ORDER BY date;
           """)

CatalogException: Catalog Error: Table with name th_skills does not exist!
Did you mean "skills_def"?

# Kdrive API

In [208]:
import pandas as pd
from dotenv import load_dotenv
import os
import io
import requests
import http
import csv
## Load dotenv variables
load_dotenv()

headers = {
'Authorization': 'Bearer ' + os.getenv("kd_token"),
'Content-Type': 'application/json',
}

def find_delimiter(text):
    sniffer = csv.Sniffer()
    delimiter = sniffer.sniff(text).delimiter
    return delimiter

kd_dns = os.getenv("kd_dns")
kd_id = os.getenv("kd_id")
conn = http.client.HTTPSConnection('api.infomaniak.com')
r = requests.get(url=f'{kd_dns}2/drive/{kd_id}/files/5476/download', headers=headers)
text = r.text
delimiter = find_delimiter(text)
df = pd.read_csv(io.StringIO(r.text), sep=';')

In [227]:
import duckdb
csvfile = io.BytesIO(r.content)
duckdb.sql("SELECT * FROM read_csv(csvfile);")

IOException: IO Error: No files found that match the pattern "csvfile"

In [254]:
skills = pd.read_csv(io.StringIO(r.text), sep=delimiter, skiprows=2)

In [256]:
skills.dropna(axis=1, how='all')

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Category Name,BACK,CATCH,...,EXPLOSIVITÄT,KOGNITIV,RUMPFKRAFT,Ballmitnahme,Dribbling,Flugball Flanken,Kopfballspiel,Passspiel,TE unter Druck,Torschuss
0,Player,Player's current team,Training Start Datetime,Training Team Name,Training Text,Training Unit Short Text,Training Unit Description,Training Unit Intensity Text,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Jara Ackermann,♀ BSC YB-Frauen,"Sep 4, 2024 10:30:00",BSC Young Boys,Individual Training,TH BiT,NaN,4 Etwas anstrengend,NaN,15.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Jara Ackermann,♀ BSC YB-Frauen,"Sep 4, 2024 10:30:00",BSC Young Boys,Individual Training,TH DS,NaN,4 Etwas anstrengend,15.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Jara Ackermann,♀ BSC YB-Frauen,"Sep 11, 2024 10:30:00",BSC Young Boys,Individual Training,TH BiT,NaN,4 Etwas anstrengend,NaN,15.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Jara Ackermann,♀ BSC YB-Frauen,"Sep 11, 2024 10:30:00",BSC Young Boys,Individual Training,TH DS,NaN,4 Etwas anstrengend,15.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
508,Felix Stuber,"BSC Young Boys, U16","Sep 16, 2024 17:30:00",NaN,THT OFF,THT OFF Z1,NaN,3 Mässig,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
509,Felix Stuber,"BSC Young Boys, U16","Sep 18, 2024 17:30:00",BSC Young Boys,Training U-16,TH HB,NaN,4 Etwas anstrengend,30.0,30.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
510,Felix Stuber,"BSC Young Boys, U16","Sep 19, 2024 17:00:00",NaN,THT OFF,TH OFF Z1,NaN,3 Mässig,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
511,Felix Stuber,"BSC Young Boys, U16","Sep 23, 2024 16:15:00",NaN,TH Training U15/16,TH RQPV,NaN,4 Etwas anstrengend,NaN,75.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [253]:
import duckdb
skills_unpivot = duckdb.sql("""
UNPIVOT skills
ON COLUMNS(* EXCLUDE ('Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 
           'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 
           'Unnamed: 6', 'Category Name'))
INTO
    NAME Kategorie
    VALUE Minuten;
           """)

BinderException: Binder Error: Column "Category Name" in EXCLUDE list not found in FROM clause

In [198]:
cursor.sql("DROP TABLE IF EXISTS skills_db;")
skills_db = cursor.sql("""CREATE TABLE IF NOT EXISTS skills_db 
           (Keeper varchar not null
           ,Team varchar not null
           ,DatumZeit varchar not null
           ,Trainingsteam varchar null
           ,Titel varchar null
           ,Übung varchar null
           ,Notizen varchar null
           ,Intensität varchar null
           ,Kategorie varchar not null
           ,Dauer real not null)""")


In [201]:
skills = kdrive.read_csv(5476)


In [203]:
cursor.sql("ALTER TABLE skills_db ALTER COLUMN DatumZeit SET DATA TYPE TIMESTAMP USING strptime(DatumZeit, '%b %-j, %Y %H:%M:%S');")

In [239]:
df = cursor.sql("SELECT * FROM postgres_db.public.goalies;").df()

In [240]:
df.head()

,Name,vorname,mannschaft,geburtsdatum,Player Name,initialen
0,Kanobel,Yanis,FE12,2013-11-26,Yanis Kanobel,YK
1,Kleeberg,Junis,FE12,2013-06-07,Junis Kleeberg,JK
2,Polo,Laurin,FE12,2013-05-14,Laurin Polo,LP
3,Röthlisberger,Roman,FE12,2013-07-29,Roman Röthlisberger,RR
4,Ung,Enea Levi,FE12,2013-05-21,Enea Levi Ung,EU
